<h1>Bitcoin in BigQuery Analaysis examples</h1>
<h2>Question 1</h2>
<h3>How many bitcoins are sent per day?</h3>


In [4]:
%%bq query --name data
#StandardSQL
SELECT o.day, SUM(o.satoshis)*0.00000001 AS btc FROM (
SELECT TIMESTAMP_MILLIS((timestamp - MOD(timestamp,86400000))) AS day, outputs.pubkey_base58 AS output_key, satoshis 
FROM 
  `bitcoin-bigquery.bitcoin.blocks` 
    JOIN
  UNNEST(transactions) AS transactions
    JOIN 
  UNNEST(transactions.outputs) AS outputs) AS o
GROUP BY day
ORDER BY day

## Visualise with Datalab commands 
http://googledatalab.github.io/pydatalab/google.datalab%20Commands.html

In [5]:
%chart line --data data --fields day,btc
title: BTC sent per day
height: 400
width: 900
hAxis:
  title: Day
vAxis:
  title: BTC volume

accessDenied: Access Denied: Table bitcoin-bigquery:bitcoin.blocks: The user 587962344783-compute@developer.gserviceaccount.com does not have permission to query table bitcoin-bigquery:bitcoin.blocks.


<h2>Question 2</h2>
<h3>How many bitcoins are sent per day?</h3>

<h2>Question 3</h2>
<h3>How many bitcoins are sent per day?</h3>

<h2>Question 4</h2>
<h3>How many bitcoins are sent per day?</h3>

<h2>Question 5</h2>
<h3>What are the associated addresses of a specific address i.e. both addresses appeared in one transaction as inputs?  </h3>
The address used here, 155fzsEBHy9Ri2bMQ8uuuR3tv1YzcDywd4, was randomly selected from the top 100 addresses on the Bitcoin Rich List.

In [1]:
%%bq query
#StandardSQL
SELECT
  DISTINCT inputs.pubkey_base58 AS associated_addresses
FROM
  `bitcoin-bigquery.bitcoin.blocks`
CROSS JOIN
  UNNEST (transactions) AS transactions
CROSS JOIN
  UNNEST (transactions.inputs) AS inputs
WHERE
  transactions.transaction_id IN (
  SELECT
    transactions.transaction_id
  FROM
    `bitcoin-bigquery.bitcoin.blocks`
  CROSS JOIN
    UNNEST (transactions) AS transactions
  CROSS JOIN
    UNNEST (transactions.inputs) AS inputs
  WHERE
    inputs.pubkey_base58 = '155fzsEBHy9Ri2bMQ8uuuR3tv1YzcDywd4')



accessDenied: Access Denied: Table bitcoin-bigquery:bitcoin.blocks: The user 587962344783-compute@developer.gserviceaccount.com does not have permission to query table bitcoin-bigquery:bitcoin.blocks.